In [1]:
#Importing necessary packages
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import folium

## Problem #1
#### Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.
<br>
<body>Answer: To address problem 1 the following procedures will be performed:
    <ol>
        <li>Use Beautiful Soup to scrape data from web</li>
        <li>Use Pandas and Numpy cleanse and present the data</li>
    </ol>
</body>

In [3]:
#Assign the URL to a variable
PageLink = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Use request library to obtain the data from URL link. Websource is a response object
WebSource = requests.get(PageLink, timeout = 5).text
#html = WebSource.content

#Parse the data from the URL, in the example below we parse it using 'lxml'
Data = BeautifulSoup(WebSource, 'lxml')

In [4]:
#Extract the HTML code enclosed in <Table>
Table = Data.table

#### Extract column headers in Table HTML tag and covert to Pandas dataframe

In [5]:
#Extract text enclosed in <tr> tag to serve as column headers
tHeaders = Table.find('tr').text

#Use '\n' as delimiters to split text which also converts to list
Headers = tHeaders.split('\n')

Headers

['', 'Postcode', 'Borough', 'Neighbourhood', '']

In [6]:
#Convert Headers list to Pandas dataframe and drop the empty columns
dfTable = pd.DataFrame(columns = Headers)
dfTable = dfTable.drop(columns = '')

dfTable

,Postcode,Borough,Neighbourhood


#### Extract data in  td HTML tag, cleanse, and covert to Pandas dataframe

In [7]:
#Extract td HTML tag in Table
tRow = Table.find_all('td')

#Convert Beautiful Soup object to string for easy cleansing
rData = str(tRow)

rData

'[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned\n</td>, <td>M2A</td>, <td>Not assigned</td>, <td>Not assigned\n</td>, <td>M3A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>\n</td>, <td>M4A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>\n</td>, <td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>\n</td>, <td>M6A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>\n</td>, <td>M6A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Lawrence_Manor" title="Lawrence Manor">Lawrence Manor</a>\n</td>, <td>M7A</td>, <td

#### Changing some characters to comma in order to have only one delimiter

In [10]:
#Replace the characters in order to have only one delimiter
Ch2Replace = ['<','>','\n'] #Characters to replace
i = 0 #iteration that resets after running all the characters to look for
Delimiter = ','
Iteration = rData #Assigning different iteration object as the for loop will alter reference object (rData)

#Convert unwanted characters to only have one delimiter
for Count in Iteration:
    for Ch2Look4 in Ch2Replace:
        if rData.find(Ch2Replace[i]) >= 0:
            LocatedChIndex = rData.find(Ch2Replace[i])
            rData = rData[:LocatedChIndex] + Delimiter + rData[LocatedChIndex + 1:]
            i += 1
        else:
            ''
    i = 0
    
#Split the data with , delimiter which also converts the data to list
RowData = rData.split(',')

#Convert the list to dataframe
TableRows = pd.DataFrame(RowData)

TableRows.head()

,0
0,[
1,td
2,M1A
3,/td
4,


#### Droping unwanted rows which contains specific words or lenght of words

In [11]:
#Drop unwanted rows using for loop

Count = 0
Iterations = TableRows.index #Assigning different iteration object as TableRows will be altered in the for loop

for Rows in Iterations:
    #Error handling, Pyhton results to an error if it was not able to find what it is searching for
    try:
        if len(TableRows.loc[Count,0]) < 3 or \
        TableRows.loc[Count,0].find('wiki') > 0 or \
        TableRows.loc[Count,0] == 'wiki' or \
        TableRows.loc[Count,0].find('=') > 0 or \
        TableRows.loc[Count,0] == '/td' or \
        TableRows.loc[Count,0].find('/td') > 0 or \
        TableRows.loc[Count,0] == '\"' or \
        TableRows.loc[Count,0].find('\"') > 0 or \
        TableRows.loc[Count,0].find('_') > 0 or\
        TableRows.loc[Count,0].find('/') > 0:
            TableRows.drop(axis = 0, index = Count, inplace = True) 
    except:
        '' #Indicates do nothing
    
    Count += 1
    
#Reset index of the dataframe for easy referencing in the future
TableRows.reset_index(drop = True, inplace = True)


#### Creating  a dataframe with three (3) columns from TableRows object with assistance from Numpy

In [13]:
#In order to determine the position of each data in the TableRows dataframe,Numpy is used to divide by batches of three. 
DataPoints = np.arange(0,TableRows.shape[0]) #creates a list of number from 0 to maximum number of rows in the dataframe
BatchCount = TableRows.shape[0]/3 #Create the number of batches
Batch = np.split(DataPoints, BatchCount) #Separate the number list (DataPoints) by batch using Numpy

Row = 0 #Row
iCol = 0 #Iteration resets to 0 after for loop used for assigning value to a column
Data = 0 #Iteration that will be used to locate a data in TableRow

Col = dfTable.columns

for Insert in Batch:
    #Determine if a data is part of a batch
    for ColValues in Batch[Row]:
        dfTable.loc[Row,Col[iCol]] = TableRows.loc[Data,0]
        iCol += 1
        Data += 1
    
    Row += 1
    iCol = 0

dfTable.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Remove the rows with "Not assigned"

In [15]:
Count = 0 #count to keep track of rows
Iteration = dfTable.index #assigning iteration to a variable as the basis[dfTable.index] of alteration is to be manipulated

for Rows in Iteration:
    if dfTable.loc[Count,'Borough'].find('assigned') >= 0:
        dfTable.drop(index = Count, inplace = True)
    elif dfTable.loc[Count,'Neighbourhood'].find('assigned') >= 0:
        dfTable.loc[Count,'Neighbourhood'] = dfTable.loc[Count,'Borough']
    Count += 1

#reset index for easy reference in the future 
dfTable.reset_index(drop = True, inplace = True)

dfTable.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [17]:
dfTable.shape[0]

210

## Problem #2
#### Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository.
<br>
<body>Answer: To address problem 2 the following procedures will be performed:
    <ol>
        <li>Use .csv provided with longitude and latitude</li>
        <li>Combined .csv file with dfTable dataframe by assigning longitude and latitude to appropriate postcode</li>
    </ol>
</body>

In [23]:
#Open csv file provided that is stored in local drive
Geolocation = pd.read_csv('C:\\Users\\ragutaya\\Geospatial_Coordinates.csv')

In [24]:
#Add new columns to dfTable dataframe
dfTable['Latitude'] = ''
dfTable['Longitude'] = ''

In [25]:
#match postcode with respective latitude and longitude
iCount = 0 #counter for reference postcode
mCount = 0 #counter for source of latitude and longitude information

#index-match equivalent for python
for Index in dfTable.index:
    for Match in Geolocation.index:
        if dfTable.loc[iCount,'Postcode'] == Geolocation.loc[mCount,'Postal Code']:
            dfTable.loc[iCount,'Latitude'] = Geolocation.loc[mCount,'Latitude']
            dfTable.loc[iCount,'Longitude'] = Geolocation.loc[mCount,'Longitude']
        else:
            ''
        mCount += 1
    
    mCount = 0 #reset counter for matching
    iCount += 1

dfTable.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,Harbourfront,43.6543,-79.3606
3,M6A,North York,Lawrence Heights,43.7185,-79.4648
4,M6A,North York,Lawrence Manor,43.7185,-79.4648


## Problem #3
#### Once you are happy with your analysis, submit a link to the new Notebook on your Github repository.
<br>
<body>Answer: To address problem 3 the following procedures will be performed:
    <ol>
        <li>Determined the longitude and latitude of Canada to create the map</li>
        <li>Used for loop to assign longitude and latitude in the created map in step 1</li>
    </ol>
</body>

In [26]:
#Coordinates of Canada
CALat = 43.761539
CALng = -79.411079
#Showing the map of Canada
MapOfCanada = folium.Map(location = [CALat, CALng], zoom_start = 10)

#Showing neighborhood of Toronto

Count = 0
for Neighbor in dfTable.Borough:
    if dfTable.loc[Count,'Borough'].find('Toronto') >= 0:
        folium.CircleMarker([dfTable.loc[Count,'Latitude'],dfTable.loc[Count,'Longitude']], radius = 1, 
                            tooltip = dfTable.loc[Count,'Neighbourhood'], 
                            fill = True, 
                            color = 'Blue', 
                            fill_color = 'Blue', 
                            fill_opacity = 0.6).add_to(MapOfCanada)
    else:
        ''
    
    Count += 1

MapOfCanada